In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from matplotlib import pyplot as plt
from sklearn import mixture
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.stats import multivariate_normal
sns.set()

In [3]:
import numpy 
numpy.__version__ 

'1.15.4'

In [2]:
np.random.seed(0)
n = 50000
n2 = 10000

In [3]:
path_train = '/home/franciscoperez/Documents/GitHub/data/BIASEDFATS/Train_rrlyr-3.csv'
path_test = '/home/franciscoperez/Documents/GitHub/data/BIASEDFATS/Test_rrlyr-3.csv'

In [4]:
data_train = pd.read_csv(path_train).sample(n)
data_test = pd.read_csv(path_test)
data = [data_test,data_train]
data = pd.concat(data)
del data_train['label']
del data_test['label']
del data['label']

In [5]:
def clean_data(data):
    try:
        data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
        data = data.drop(['Pred', 'Pred2', 'h', 'e', 'u', 'ID'], axis=1)
    except:
        print('---')
    names = data.columns
    scaler = preprocessing.StandardScaler()
    data = scaler.fit_transform(data)
    data = pd.DataFrame(data, columns=names)
    return data

def kl_divergence(p, q):
    epsilon = 0.00001
    p = p + epsilon
    q = q + epsilon
    kl = p * np.log(p / q)
    return kl

def pca_reduction(data, components=4):
    pca = PCA(n_components=components)
    pca.fit(data)
    data = pca.transform(data)
    data = pd.DataFrame(data)
    return data

In [6]:
data_test = clean_data(data_test)
data = clean_data(data)

In [7]:
data_test = pca_reduction(data_test)
data = pca_reduction(data)

In [8]:
clf = mixture.GaussianMixture(n_components=10, covariance_type='spherical', max_iter=1000)
gmm_test = clf.fit(data_test)
clf_survey = mixture.GaussianMixture(n_components=10, covariance_type='spherical', max_iter =1000)
gmm_survey = clf_survey.fit(data)

        'full' (each component has its own general covariance matrix),
        'tied' (all components share the same general covariance matrix),
        'diag' (each component has its own diagonal covariance matrix),
        'spherical' (each component has its own single variance).

In [9]:
means_test = gmm_test.means_
covariances_test = gmm_test.covariances_
weights_test = gmm_test.weights_

In [10]:
means_survey = gmm_survey.means_
covariances_survey = gmm_survey.covariances_
weights_survey = gmm_survey.weights_

In [11]:
def separated_gaussians(means, covariances):
    gauss = []
    for j in range(means.shape[0]):
        gmm = multivariate_normal(mean=means[j], cov=covariances[j])
        gauss.append(gmm)
    return gauss

In [12]:
gaussians_test = separated_gaussians(means_test, covariances_test)
gaussians_survey = separated_gaussians(means_survey, covariances_survey)

In [13]:
def gmm_pdf(x, gaussians, weights):
    for i in range(weights.shape[0]):
        pdf =+ weights[i]*gaussians[i].pdf(x)
    return pdf

In [14]:
kl = 0
data_sample = data_test.sample(n2)
for j in range(data_sample.shape[0]):
    p = gmm_pdf(data.iloc[j], gaussians_test, weights_test)
    q = gmm_pdf(data.iloc[j], gaussians_survey, weights_survey)
    kl = kl+kl_divergence(p, q)
print('KL', kl)

KL -0.46118958460126325
